# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [26]:
import numpy as np
import pandas as pd
import scipy.stats as st

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [16]:
pokemon.pivot_table(index=['Legendary'],values=['Name'],aggfunc='count')

,Name
Legendary,
False,735
True,65


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [24]:
leg=pokemon[pokemon['Legendary']==True]
non_leg=pokemon[pokemon['Legendary']==False]
print('Legendary')
print(leg['Total'].mean())
print(leg['Total'].std())
print('\nNon-Legendary')
print(non_leg['Total'].mean())
print(non_leg['Total'].std())

Legendary
637.3846153846154
60.93738905315346

Non-Legendary
417.21360544217686
106.76041745713022


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [28]:
st.ttest_ind(leg['Total'],non_leg['Total'],equal_var=False)

Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957446e-47)

What do you conclude from this test? Write your conclusions below.

In [ ]:
#Since the p-value is extremely low, we can safely reject the null hypothesis that the two groups are statistically equal. 

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [29]:
pokemon.pivot_table(index=['Type 1'],values=['Name'],aggfunc='count')

,Name
Type 1,
Bug,69
Dark,31
Dragon,32
Electric,44
Fairy,17
Fighting,27
Fire,52
Flying,4
Ghost,32


Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [45]:
water=pokemon[pokemon['Type 1']=='Water']
not_water=pokemon[pokemon['Type 1']!='Water']
print('Water')
print(water['Total'].mean())
print(water['Total'].std())
print('\nAll Others')
print(not_water['Total'].mean())
print(not_water['Total'].std())

Water
430.45535714285717
113.18826606431458

All Others
435.85901162790697
121.09168230208066


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [48]:
st.ttest_ind(water['Total'],not_water['Total'],equal_var=True)

Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

Write your conclusion below.

In [ ]:
#We cannot reject the null hypothesis, since the p-value is too large

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [50]:
st.ttest_rel(pokemon['Attack'],pokemon['Defense'])

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

Describe the results of the test in the cell below.

In [ ]:
#I reject the null hypothesis that the mean defense and attack scores are equal, due to very low p-value

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [51]:
st.ttest_rel(pokemon['Sp. Def'],pokemon['Sp. Atk'])

Ttest_relResult(statistic=-0.853986188453353, pvalue=0.3933685997548122)

Describe the results of the test in the cell below.

In [ ]:
#We cannot reject the null hypothesis that the mean of special defense and attack are equal 

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [54]:
st.ttest_1samp(pokemon['Sp. Def']-pokemon['Sp. Atk'],0)

Ttest_1sampResult(statistic=-0.853986188453353, pvalue=0.3933685997548122)

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [60]:
table=pd.crosstab(pokemon['Legendary'],pokemon['Type 1']=='Water')

Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [62]:
st.chi2_contingency(table)

(2.9429200762850503,
 0.08625467249550949,
 1,
 array([[632.1, 102.9],
        [ 55.9,   9.1]]))

Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
#We should not reject the null hypothesis, since p-value is 8%. 